# Introduction

[Automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) is a method for numerically finding the derivative of a function at a given point. It can be used to find derivatives of complex functions where computing the symbolic derivative can be impossible or computationally costly.

Automatic differentiation is more accurate than other numerical differentiation methods such as the [finite difference method](https://en.wikipedia.org/wiki/Finite_difference_method). The finite difference method attempts to find the derivative of a function at a given point by adding a small perturbation ($\epsilon$):

$$ \frac{\partial f}{\partial x} \approx \frac{f(x+\epsilon) - f(x)}{\epsilon} $$

When the perturbation is too large, the estimate for the derivative is not accurate. When the perturbation is too small, it starts to amplify floating point errors. 

Automatic differentiation achieves high accuracy while avoiding amplified floating point errors by (1) breaking down the function into a sequence of elementary functions (e.g., sin, cos, log, and exp), (2) calculating the exact derivation of these elementary functions, (3) and finally combining them using the [chain rule](https://en.wikipedia.org/wiki/Chain_rule), the [product rule](https://en.wikipedia.org/wiki/Product_rule), and simple mathematical operations (such as addition and multiplication). 

Given its speed and precision, automatic differentiation is popular within the field of computational science where it has numerous applications. This software package as an implementation of automatic differentiation using Python.

---

# Background

## Motivation

Automatic differentiation allows us to compute the true analytic derivative of a function to machine precision. At a high level, this is done by breaking down complex functions into their elementary components and propogating their derivative via the chain rule. Other methods of finding derivatives include symbolic differentiation and finite fifference. Symbolic differentiation is also accurate to machine precision but is more computationally costly and its implementation is more complex. Finite difference is easier and less costly to implement but can quickly lead to floating point errors. As such, automatic differentiation is a more precise and lightweight methodology to use.

## Some Calculus

The [product rule](https://en.wikipedia.org/wiki/Product_rule) is used to find the derivative of the product of two or more functions. In its simplest form, if $f$ and $g$ are functions, the derivative of their product is given by the following equation: 

$$ [f(x)g(x)]' = f'(x)g(x)+f(x)g'(x) $$

The chain rule is used for computing the derivative of the composition of two or more functions. In its simplest form, if $f$ and $g$ are functions, the derivative of their composition is given by the following equation:

$$ [f(g(x))]' = f'(g(x))*g'(x) $$

## Modes of Automatic Differentiation

The automatic differentiation method can be implemented in two ways depending on how the chain rule is utilized. Consider the formulation:

$$ f(x) = g_3(g_2(g_1(x))) $$

Using the chain rule, this function's derivative at point $x = a$ is computed as:

$$ f'(a) = g_3'(.)*g_2'(.)*g_1'(a) $$

The forward mode of automatic differentiation recursively propagates the calculated derivative from the right: first calculates $g_1'(a)$, then $g_2'(.)$, then $g_3'(.)$, and so on. 

The reverse mode of automatic differentiation recursively propagates the calculated derivative from the left: first calculates $g_3'(.)$, then $g_2'(.)$, then $g_1'(a)$, and so on.

In our implementation, we will focus on the forward mode. 

## Forward Mode

A useful tool associated with the forward mode is the computational trace. Using the computational trace, we can list the steps required to go from input values (the point at which the derivative is evaluated) to the input function.  

Consider the following function:

$$ f(x) = sin(e^{2x}) $$

Say we want to evaluate the derivative of this function at $x = 5$. The steps for calculating the derivative using forward mode are given in the following table:

| Trace | Elementary Function | Function Value | Derivative | Derivative Value |
| :------: | :----------: | :-------: | :---------: | :--------: |
| $x_{1}$ | $x$ | 5 | $\dot{x}_{1}$ | $1$ |
| $x_{2}$ | $2x_{1}$ | $10$ | $2\dot{x}_{1}$ | $2$ |
| $x_{3}$ | $e^{x_{2}}$ | $e^{10}$ | $e^{x_{2}}\dot{x}_{2}$ | $2e^{10}$ |
| $x_{4}$ | $sin(x_{3})$ | $sin(e^{10})$ | $cos(x_{3})\dot{x}_{3}$ | $2e^{10}cos(e^{10})$ |

We get the required derivative value $2e^{10}cos(e^{10})$.

## Dual Numbers

We utilize [dual numbers](https://en.wikipedia.org/wiki/Dual_number) in our implementation of the forward mode of automatic differentiation. Dual numbers are an extension to real numbers (similar to complex numbers). Dual numbers introduce a new element (typically represented by $\epsilon$) with the useful property $\epsilon^2 = 0$.  Using dual numbers and [Taylor series](https://en.wikipedia.org/wiki/Taylor_series), we can find the derivative of a function quickly. 

Say we have a function $f$ and we want to find its derivative at point $x = a$. We will set $x = a + b\epsilon$ and find the Taylor expansion:

$$ f(a+b\epsilon) = \sum_{n=0}^{\infty} \frac{(b\epsilon)^nf^{(n)}(a)}{n!} = f(a) + b\epsilon f'(a) $$

All higher-order terms are equal to $0$ because of the dual number property $\epsilon^2 = 0$. For function $f$, we get its derivative at point $x = a$ directly from the second term in the Taylor expansion.

## Root Finding
When the root of a function $f: \mathbb{R}^n \to \mathbb{R}^m$ is sought, we generally must use a root finding algorithm.
Many functions have roots that can not be found analytically, or are simply too complicated to be able to find the root by hand. 
We therefore must resort to solving them computationally.
One of the most popular methods for this is Newton's method.
Newton's method for root finding incorporates gradient information so it is a perfect candidate to be implemented using automatic differentiation.
The algorithm is as follows for a function $\textbf{f}$, :

> initialize $\textbf{x}$ \\
> for $k = 0, 1, 2, \cdots $
>> $\textbf{x}_{k+1} = \textbf{x}_k - \textbf{J}_{f}^{-1}\textbf{f}(\textbf{x}_k)$

This relatively simple algorithm is quite powerful, provided that the initial guess is close enough to the root.
Newton's method fails when the derivative is 0, or when the initial guess is bad enough such that the current guess is farther from the root than the previous step.

## Optimization
Optimization is of great interest in many fields such as machine learning, physics, and engineering.
There are many optimization algorithms out there, so we will only touch on a few.
The algorithms we have implemented are all for functions $f: \mathbb{R}^n \to \mathbb{R}$.
We have chosen algorithms that require gradients and first derivative knowledge only.
The first algorithm used is the classic gradient descent, seen below:
> initialize $\textbf{x}_0$ \\
> for $k = 0, 1, 2, \cdots$ \\
>> $\textbf{x}_{k+1} = \textbf{x} - \alpha_k \nabla f(\textbf{x}_k)$ \\

where $\alpha_k$ is the step size parameter.
Gradient descent is quite popular due to its simplicity, however its convergence tends to be quite slow.

The next algorithm implemented is the BFGS algorithm.
This algorithm approximates the Hessian with gradient informatation, and is similar to Newton's method in the use of Hessian. It is as follows:

> initalize $\textbf{x}_0$ \\
> initialize $\textbf{B}_0$ hessian estimate \\
> for $k = 0, 1, 2, \cdots $ \\
>> Solve $\textbf{B}_k\textbf{s}_k = -\nabla f(\textbf{x}_k)$ for $\textbf{s}_k$ \\
>> $\textbf{x}_{k+1} = \textbf{x}_k + \textbf{s}_k$ \\
>> $\textbf{y}_k = \nabla f(\textbf{x}_{k+1}) - \nabla f(\textbf{x})$ \\
>> $\textbf{B}_{k+1} = \textbf{B}_k + (\textbf{y}_k\textbf{y}_k^T)/(\textbf{y}_k^T\textbf{s}_k) - (\textbf{B}_k\textbf{s}_k\textbf{s}_k^T\textbf{B}_k)/(\textbf{x}_k^T\textbf{B}_k\textbf{s}_k)$

This method performs similarly to Newton's method for root finding, although is slower and less accurate due to Hessian approximation.

The last method implemented is the conjugate gradient method.
This method aims to approximate the Hessian without storing the approximation directly. It is as follows:
> initialize $\textbf{x}_0$ \\
> $\textbf{g}_0 = \nabla f(\textbf{x}_0)$ \\
> $\textbf{s}_0 = -\textbf{g}_0$ \\
> for $k = 0, 1, 2, \cdots$ \\
>> $\textbf{x}_{k+1} = \textbf{x}_k + \alpha_k \textbf{s}_k$ \\
>> $\textbf{g}_{k+1} = \nabla f(\textbf{x}_{j+1})$ \\
>> $\beta_{k+1} = (\textbf{g}_{k+1}^T\textbf{g}_{k+1})/(\textbf{g}_k^T \textbf{g}_k)$ \\
>> $\textbf{s}_{k+1} = -\textbf{g}_{k+1} + \beta_{k+1}\textbf{s}_k$ \\


This method performs similar to BFGS, although slightly worse.

---

# How To Use

## Installation via GitHub:

Download package and install dependencies:

*   `conda create -n test python=3.6`
*   `conda activate test`
*   `git clone https://github.com/make-AD-ifference/cs207-FinalProject.git`
*   `cd cs207-FinalProject/`
*   `pip install -r requirements.txt`
*   `pytest`

## Installation via PyPI:

Download package (dependencies automatically installed):

*   `conda create -n test python=3.6`
*   `conda activate test`
*   `pip install autodiff-kgl`

## Importing the package in Python (an example):

In [ ]:
from autodiff.autodiff import AutoDiff
AD1 = AutoDiff(2,3)
AD2 = AutoDiff.cos(AD1)
AD3 = AutoDiff(5,[1,2])

import autodiff.optimization
import autodiff.root_finding
import autodiff.plot

In case you get ModuleNotFoundError, please ensure you are using the same virtual environmnet where you ran the pip install command and using the Python interpreter from that virtual environment.

## Packaging Discussion
In thinking through how to use our software package, we considered the following:

#### Who are our users?
We decided to target somewhat tech-savvy users who would be comfortable with a command line application as opposed to a more approachable and user-friendly GUI. Any single user should be able to install the package easily onto their machine without requiring broader deployment. 

#### Where should it run? 
We expect our package to be installed and run on any desktop devices through the command line. It should work regardless of the operating system as long as the user has Python on their machine (our assumption is that most users will have Python pre-installed on their Mac or Linux machines or will be able to easily download it otherwise).

## Packaging Choice
We plan to use pip as the package manager. We considered some other options such as conda install but ultimately chose pip because Python developers are already familiar with it and we did not want the installation process to be a barrier to using our program. We plan to register the name of our package on PyPI and upload source distributions via the setup.py file. To ensure that our package can be downloaded regardless of the user’s Python version, we plan to use pip instead of pip3. 

---

# Software Organization


## Directory Structure

The directory structure looks like:

`make-AD-ifference/`

>`setup.py`

>`.gitignore`

>`.travis.yml`

>`.requirements.txt`

>`coverage.txt`

>`README.md`

>`LICENSE`

>`autodiff/`

>>`README.md`

>>`__init__.py`

>>`autodiff.py`

>>`optimization.py`

>>`root_finding.py`

>>`plot.py`

>`docs/`

>>`documentation.ipynb`

>>`milestone1.ipynb`

>>`milestone2.ipynb`

>`tests/`

>>`README.md`

>>`test_autodiff.py`

>>`test_optimization.py`

>>`test_root_finding.py`

>>`test_plot.py`

>`scratch/`


**Note: Scratch folder contains scripts we wrote but did not end up using in our final implementation.**


## Modules
- `autodiff.py`: contains the autodifferentiation class `AutoDiff`, which implements Forward Mode AD. This module serves as our custom library that allows users to evaluate functions and their derivatives for each input value. The class also includes custom methods that our program supports. Each method returns the value and derivative of the specified function. See the implementation section for more details. 

- `optimization.py`: contains the algorithms for gradient descent method, BFGS method, and conjugate gradient method. Optimization only works for scalar to scalar and vector to scalar functions. 

- `root_finding.py`: contains newton's method for root finding. Root finding works for all vector to vector and scalar to scalar functions.  

- `plot.py`: contains the plotting routine which uses the trace on top of a contour plot of the function. 



## Testing

All testing is done using TravisCI, and evaluation of the tests is done using CodeCov. Our main test suite `test_autodiff.py` and extension test suites live in the `tests/` folder.

- `test_autodiff.py`: contains our test suite for the `autodiff.py` module. It includes tests for scalar and vector functions.

- `test_optimization.py`: contains our test suite for the `optimization.py` module.

- `test_root_finding.py`: contains our test suite for the `root_finding.py` module.


---


# Implementation


## Class: AutoDiff

We are using a simple one-class structure to implement forward mode autodifferentiation. The AutoDiff class keeps track of the trace value and the derivative of functions for each input variable. It has two attributes `val` and `der` which represent the value and the derivative respecitvely. We have a set of elementary functions that can easily be expanded. These elementary functions serve as building blocks for users to assemble the function they wish to evaluate.
We provide users with an understandable interface that conveys which elementary functions our software supports. If a user enters a function that is a combination of any of these, our program is able to handle the input. Otherwise, we return a descriptive error and allow users to enter a new function.

Elementary functions defined in the AutoDiff class:
- Ln (natural log)
- Log 
- Exp
- Exp with any base (e.g. 2 ^ x)
- Sin
- Cos
- Tan
- arcsine
- arccosine
- arctangent
- sinh
- cosh
- tanh
- Sqrt
- logistic

Elementary mathematical operations supported (note: the cummutative properties of operations are preserved):
- Addition
- Subtraction
- Multiplication
- Division
- Power

Comparison operators:
- equal
- not equal
- greater or equal
- greater
- less or equal
- less

Unary operator:
- Negative



## More Details about Attributes & Methods
We have overloaded the methods in class `AutoDiff` to give the user flexibility in how functions are entered. Overloaded functions support elementary operations between two `AutoDiff` objects or 1 `AutoDiff` object and 1 scalar. This way, we also preserve the cummutative nature of functions where needed. As an example, the user may input f = 2x + 3 or f = 3 + 2x. Regardless of the order, the program returns the correct value and derivative. 
Each `AutoDiff` object has as attributes the value and the derivative, calculated using trace variables and elementary operations, for a given input value. 

More on the custom operations and functions supported in class `AutoDiff`:
- `__init__`:  constructor of class `AutoDiff`. Initializes an `AutoDiff` object, setting `self.der` initially to 1. Takes in an input value `val` at which to evaluate the function value and derivative.
- `__str__`: returns the string value of the function.
- `__repr__`: returns the string value of the function.
- `__eq__`: returns boolean, True if value of self is equal to other else False.
- `__ne__`: returns boolean, True if value of self is not equal to other else False.
- `__gt__`: returns boolean, True if value of self is greater than or equal to other else False.
- `__ge__`: returns boolean, True if value of self is greater than other else False.
- `__lt__`: returns boolean, True if value of self is less than or equal to other else False.
- `__le__`: returns boolean, True if value of self is less than other else False.
- `__add__`: overloaded addition function. Supports adding two `AutoDiff` objects or 1 `AutoDiff` object and 1 scalar.
- `__radd__`: Supports addition of two `AutoDiff` objects or 1 `AutoDiff` object and 1 scalar regardless of input order. Ensures cummutative property of addition is preserved.
- `__sub__`: overloaded subtraction function. Supports subtraction between two `AutoDiff` objects or 1 `AutoDiff` object and 1 scalar.
- `__rsub__`: Supports subtraction of the form scalar - `AutoDiff` instead of `AutoDiff` - scalar.
- `__mul__`: overloaded multiplication function. Supports multiplying two `AutoDiff` objects or 1 `AutoDiff` object and 1 scalar.
- `__rmul__`: overloaded multiplication function. Supports multiplying two `AutoDiff` objects or 1 `AutoDiff` object and 1 scalar regardless of input order. Ensures cummutative property of multiplication is preserved.
- `__truediv__`: overloaded division function. Supports dividing an `AutoDiff` object by another `AutoDiff` object or an `AutoDiff` object by a scalar.
- `__rtruediv__`: Supports division of the form scalar / `AutoDiff` instead of `AutoDiff` / scalar.
- `__pow__`: overloaded power function. Supports an `AutoDiff` object to the power of another `AutoDiff` object or an `AutoDiff` object to the power of a scalar.
- `__rpow__`: overloaded power function. Supports an `AutoDiff` object to the power of another `AutoDiff` object or a scalar to the power of an `AutoDiff` object.
- `__neg__`: returns negated `AutoDiff` object.
- `sin`: returns sine of `AutoDiff` object.
- `cos`: returns cosine of `AutoDiff` object.
- `tan`: returns tangent of `AutoDiff` object.
- `arcsine`: returns arcsine of `AutoDiff` object.
- `arccosine`: returns arccosine of `AutoDiff` object.
- `arctangent`: returns arctangent of `AutoDiff` object.
- `sinh`: returns sinh of `AutoDiff` object.
- `cosh`: returns cosh of `AutoDiff` object.
- `tanh`: returns tanh of `AutoDiff` object.
- `ln`: returns natural log of `AutoDiff` object.
- `log`: returns log of `AutoDiff` object with base `base` as second input.
- `exp`: returns exponential of `AutoDiff` object.
- `expm`: returns exponential of `AutoDiff` object with base `base` as second input.
- `logistic`: returns logistic of `AutoDiff` object.
- `sqrt`: returns square root of `AutoDiff` object.

## Using AutoDiff
- Begin by initializing an AutoDiff object with a given value and derivative with respect to each variable: ( # variables = # columns in second argument)

`x = AutoDiff(2, [3 , 1])`

- Then, define the function in the following manner:

`f = x ** 2`

`f = AutoDiff.sin(x)`

`f = AutoDiff.log(x, 2)`

- The function's value can then be accessed as `f.val`
- The function's derivative can be accessed as `f.der`

## Using Root Finding

The use of the Newton's method for root finding is quite simple. 
We simple declare the variables of interest, write a lambda function, and put those into the root finding routine.
The function ouput must be a vector, even a single element vector, for the root finding to work.

`x = AutoDiff(2, [1., 0.])`

`y = AutoDiff(1, [1., 0.])`

`func = lambda x: [x[0]**3 + x[1]**3]`

`output = newton(func, [x, y])`

## Using Optimization

The optimization routines are similarly straightforward.
The only significant change here, is that functions to be optimized return a scalar instead of a vector.

`x = AutoDiff(2, [1., 0.])`

`y = AutoDiff(1, [1., 0.])`

`func = lambda x: 100*(x[1] - x[0]**2)**2 + x[0]**2`

`output = BFGS(func, [x, y])`

## Plotting

The plotting script is designed to work directly with the output of the root finding or optimization routines.
With the output of the optimization routine, we simply use the plotting function as
`plot(func, output[3], output[1])`
where `func` is the function we minimized.

## External Dependencies
- `Numpy`: In order to run our program, users have to import the numpy library as follows:
`import numpy as np`. The Numpy library provides support for the elementary mathematical functions and operations handled by our program.

- `matplotlib`: In order to use our plotting functionality, users have to import the numpy library as follows:
`import matplotlib`. The matplotlib library provides support for plotting in Python.

## Efficiency

We have to consider things such as memory accesses, which can greatly speed up or slow down the code. We also have to consider numerical precision, although this shouldn’t be an issue with our hard-coding of functions and derivatives. Another possible consideration is memory overhead. Efficient storage of the functions and evaluations is crucial to making the code usable.


---


## Vector Valued Functions

Our package now handles vector valued functions.
Generally, we will be able to handle functions of the form
$$ f: \mathbb{R}^m \to \mathbb{R}^n $$
for arbitrary $m$ and $n$.
In order to accomplish this we will need the Jacobian matrix
$$\textbf{J} = \left[\begin{matrix} \frac{\partial f_1}{\partial x_1} & \cdots  & \frac{\partial f_1}{\partial x_n} \\
\vdots & \ddots & \vdots \\
\frac{\partial f_m}{\partial x_1} & \cdots & \frac{\partial f_m}{\partial x_n}  \end{matrix} \right]$$


## Our Extensions

The extension is a root-finding and optimization suite, along with plotting functionality.
For both procedures we implemented strictly gradient based methods.
Methods such as Newton's method for optimization were excluded because they require second derivative information.
Similarly, methods like the bisection method for root-finding were excluded because they required no gradient information.
This left us with a fairly small subset of root-finding and optimization functions.
An additional plotting tool is included to aid in visualization of the algorithms.


For root-finding, we have Newton's method.
For optimization, we have the gradient descent, BFGS, and conjugate gradient methods.
In order to have standard Python usability, these methods are stored in
`autodiff/root_finding.py` and `autodiff/optimization.py`.
This structure allows us to call optimization methods as `autodiff.optimize.conjugat_gradient()`.

The input syntax for each method is relatively straighforward, and based on the vector function implementation. In order to call an optimization routine, the user must simply define the variables of interest and function as such

`x = ad(x, [1., 0.])`

`y = ad(y, [0., 1.])`

`fn = lambda x: x[0]**2 + y[0]**2`

`output = autodiff.optimize.gradient_descent(fn, [x, y])`

Paired with these, we have a plotting script.
It takes in an optimization or root finding trace and plots that as a scatter plot over a filled contour plot of the function.
If the routine ocnverged, a black star is plotted at the converged point. If not, no black star is plotted.
An example of the conjugate gradient function plotted on top of the Rosenbrock function $f(x,y) = 100(y-x^2)^2 + (1-x)^2$ is seen below, both with and without convergence.

Convergence

![graph](https://drive.google.com/uc?id=1fWQyum_CiN8CJMIH1Iq6KKwi51wePg5v)


\
\
No Convergence

![graph](https://drive.google.com/uc?id=1oKQjvTnBgb64zNjqFiiISJNY0GzKcHVL)

